In [1]:
from bs4 import BeautifulSoup
import re
import pandas as pd
import requests
import glob

In [2]:
api_key = 'd38276c0' #key for sending requests to http://www.omdbapi.com

In [3]:
def getTitleAndText(strongString):
    found = re.sub('<strong title=', '', strongString)
    found = re.sub('</strong>', '', found)
    spl = found.split(">")
    return (spl[0].strip("\""), spl[1])

def getId(link):
    return link.split("/")[2]

In [4]:
with open('imdb_most_popular_movies_dump.html', 'r') as f:

    contents = f.read()
    soup = BeautifulSoup(contents, 'lxml') #html.parser

    ids = []
    images = [];
    titlemeters = []
    titlemeterCounts = []

    ranks = []

    ratings = []
    ratingsBased = []

    posterElem = soup.find_all("td", {"class": "posterColumn"})
    for tag in posterElem:
        ids.append(getId(tag.find('a', href=True)['href']))
        images.append(tag.find('img')['src'])
       
    # get direction of titlemeter 'up' or 'down'
    titlemeterElem = soup.find_all("span", {"class": "titlemeter"})
    for tag in titlemeterElem:
        strTag = str(tag)
        tt = re.search(r'up|down', strTag)
        titlemeters.append(strTag[tt.span()[0] : tt.span()[1]])
    
    velocityElem = soup.find_all("div", {"class": "velocity"})
    for tag in velocityElem:
        rankAndTitlemeterCount = tag.text.replace('\n','').replace(')','').split("(")
        ranks.append(rankAndTitlemeterCount[0])
        titlemeterCounts.append(rankAndTitlemeterCount[1])        
    

    ratingElem = soup.find_all("td", {"class": "imdbRating"})
    for tag in ratingElem:
        strongTag = tag.find('strong')
        if strongTag:
            ratingBased, rating = getTitleAndText(str(strongTag))
            ratings.append(rating)
            ratingsBased.append(ratingBased)
        else:
            ratings.append('N/A')
            ratingsBased.append("N/A")

    data = list(zip(ranks, ids, images, titlemeterCounts, titlemeters,ratings, ratingsBased))
    cols = ["Rank", "Id", "Image", "Titlemeter count", "Titlemeter", "Rating", "Rating based on"]
    df1 = pd.DataFrame(data, columns=cols)
#     print(df1)

In [5]:
responses = [];

for identifier in ids:
    url = 'http://www.omdbapi.com/?i='+identifier+'&apikey='+api_key
    response = requests.get(url)
    if response.status_code == 200:
        json_object = response.json()
        if json_object:
            responses.append(json_object)
    else:
        print("Response return error: ", response.status_code)
          
df2 = pd.DataFrame(responses)


In [7]:
result = pd.concat([df1, df2], axis=1, join="inner")
result

,Rank,Id,Image,Titlemeter count,Titlemeter,Rating,Rating based on,Title,Year,Rated,...,Metascore,imdbRating,imdbVotes,imdbID,Type,DVD,BoxOffice,Production,Website,Response
0,1,tt11161474,https://m.media-amazon.com/images/M/MV5BMGE3Mz...,4,up,7.1,"7.1 based on 15,722 user ratings",Pieces of a Woman,2020,R,...,66,7.1,"13,523",tt11161474,movie,N/A,N/A,N/A,N/A,True
1,2,tt2948372,https://m.media-amazon.com/images/M/MV5BZGE1MD...,1,down,8.1,"8.1 based on 139,955 user ratings",Soul,2020,PG,...,86,7.8,662,tt2948372,movie,N/A,N/A,N/A,N/A,True
2,3,tt7126948,https://m.media-amazon.com/images/M/MV5BNWY2NW...,1,down,5.4,"5.4 based on 137,696 user ratings",Wonder Woman 1984,2020,PG-13,...,60,5.5,"129,163",tt7126948,movie,N/A,"$32,600,000","DC Comics, Warner Bros., DC Entertainment, Atl...",N/A,True
3,4,tt6723592,https://m.media-amazon.com/images/M/MV5BYzg0NG...,1,down,7.5,"7.5 based on 271,311 user ratings",Tenet,2020,PG-13,...,69,7.5,"262,974",tt6723592,movie,N/A,"$57,929,000","Syncopy, Warner Bros. Pictures",N/A,True
4,5,tt9130508,https://m.media-amazon.com/images/M/MV5BYTY3Nz...,116,up,N/A,N/A,Cherry,2021,R,...,N/A,N/A,N/A,tt9130508,movie,N/A,N/A,N/A,N/A,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,tt0109830,https://m.media-amazon.com/images/M/MV5BNWIwOD...,4,down,8.8,"8.8 based on 1,804,075 user ratings",Forrest Gump,1994,PG-13,...,82,8.8,"1,801,266",tt0109830,movie,N/A,"$330,455,270",Paramount Pictures,N/A,True
96,97,tt4263482,https://m.media-amazon.com/images/M/MV5BMTUyNz...,51,up,6.9,"6.9 based on 200,714 user ratings",The Witch,2015,R,...,83,6.9,"201,120",tt4263482,movie,N/A,"$25,138,705",N/A,N/A,True
97,98,tt4154756,https://m.media-amazon.com/images/M/MV5BMjMxNj...,35,down,8.4,"8.4 based on 830,402 user ratings",Avengers: Infinity War,2018,PG-13,...,68,8.4,"828,791",tt4154756,movie,N/A,"$678,815,482",Marvel Studios,N/A,True
98,99,tt8046498,https://m.media-amazon.com/images/M/MV5BZmIzOW...,845,up,5.9,5.9 based on 669 user ratings,Don't Tell a Soul,2020,R,...,N/A,7.9,55,tt8046498,movie,N/A,N/A,N/A,N/A,True


In [8]:
#investigate what ratings were listed
#create separate columns for each rating, when not available, put 'N/A' value
sources_set = set()
for rating in df2['Ratings']:
    if len(rating) > 0:
        for elem in rating:
            sources_set.add(elem['Source'])
# print(list(sources_set))

sources = [];
for rating in df2['Ratings']:
    if len(rating) > 0:
        for elem in rating:
            unique_ratings = {};
            for source in sources_set:
                if source == elem['Source']:
                    unique_ratings[source] = elem['Value']
                else:
                    unique_ratings[source] = 'N/A'
    else:
        for source in sources_set:
            unique_ratings[source] = 'N/A'
    sources.append(unique_ratings)

print(sources)

[{'Internet Movie Database': 'N/A', 'Metacritic': '66/100', 'Rotten Tomatoes': 'N/A'}, {'Internet Movie Database': 'N/A', 'Metacritic': '86/100', 'Rotten Tomatoes': 'N/A'}, {'Internet Movie Database': 'N/A', 'Metacritic': '60/100', 'Rotten Tomatoes': 'N/A'}, {'Internet Movie Database': 'N/A', 'Metacritic': 'N/A', 'Rotten Tomatoes': 'N/A'}, {'Internet Movie Database': 'N/A', 'Metacritic': 'N/A', 'Rotten Tomatoes': 'N/A'}, {'Internet Movie Database': 'N/A', 'Metacritic': 'N/A', 'Rotten Tomatoes': 'N/A'}, {'Internet Movie Database': 'N/A', 'Metacritic': '72/100', 'Rotten Tomatoes': 'N/A'}, {'Internet Movie Database': 'N/A', 'Metacritic': '46/100', 'Rotten Tomatoes': 'N/A'}, {'Internet Movie Database': 'N/A', 'Metacritic': '74/100', 'Rotten Tomatoes': 'N/A'}, {'Internet Movie Database': 'N/A', 'Metacritic': 'N/A', 'Rotten Tomatoes': 'N/A'}, {'Internet Movie Database': 'N/A', 'Metacritic': 'N/A', 'Rotten Tomatoes': 'N/A'}, {'Internet Movie Database': 'N/A', 'Metacritic': 'N/A', 'Rotten Toma

In [9]:
df3 = pd.DataFrame(sources)

result = pd.concat([result, df3], axis=1)
result

,Rank,Id,Image,Titlemeter count,Titlemeter,Rating,Rating based on,Title,Year,Rated,...,imdbID,Type,DVD,BoxOffice,Production,Website,Response,Internet Movie Database,Metacritic,Rotten Tomatoes
0,1,tt11161474,https://m.media-amazon.com/images/M/MV5BMGE3Mz...,4,up,7.1,"7.1 based on 15,722 user ratings",Pieces of a Woman,2020,R,...,tt11161474,movie,N/A,N/A,N/A,N/A,True,N/A,66/100,N/A
1,2,tt2948372,https://m.media-amazon.com/images/M/MV5BZGE1MD...,1,down,8.1,"8.1 based on 139,955 user ratings",Soul,2020,PG,...,tt2948372,movie,N/A,N/A,N/A,N/A,True,N/A,86/100,N/A
2,3,tt7126948,https://m.media-amazon.com/images/M/MV5BNWY2NW...,1,down,5.4,"5.4 based on 137,696 user ratings",Wonder Woman 1984,2020,PG-13,...,tt7126948,movie,N/A,"$32,600,000","DC Comics, Warner Bros., DC Entertainment, Atl...",N/A,True,N/A,60/100,N/A
3,4,tt6723592,https://m.media-amazon.com/images/M/MV5BYzg0NG...,1,down,7.5,"7.5 based on 271,311 user ratings",Tenet,2020,PG-13,...,tt6723592,movie,N/A,"$57,929,000","Syncopy, Warner Bros. Pictures",N/A,True,N/A,N/A,N/A
4,5,tt9130508,https://m.media-amazon.com/images/M/MV5BYTY3Nz...,116,up,N/A,N/A,Cherry,2021,R,...,tt9130508,movie,N/A,N/A,N/A,N/A,True,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,tt0109830,https://m.media-amazon.com/images/M/MV5BNWIwOD...,4,down,8.8,"8.8 based on 1,804,075 user ratings",Forrest Gump,1994,PG-13,...,tt0109830,movie,N/A,"$330,455,270",Paramount Pictures,N/A,True,N/A,82/100,N/A
96,97,tt4263482,https://m.media-amazon.com/images/M/MV5BMTUyNz...,51,up,6.9,"6.9 based on 200,714 user ratings",The Witch,2015,R,...,tt4263482,movie,N/A,"$25,138,705",N/A,N/A,True,N/A,83/100,N/A
97,98,tt4154756,https://m.media-amazon.com/images/M/MV5BMjMxNj...,35,down,8.4,"8.4 based on 830,402 user ratings",Avengers: Infinity War,2018,PG-13,...,tt4154756,movie,N/A,"$678,815,482",Marvel Studios,N/A,True,N/A,68/100,N/A
98,99,tt8046498,https://m.media-amazon.com/images/M/MV5BZmIzOW...,845,up,5.9,5.9 based on 669 user ratings,Don't Tell a Soul,2020,R,...,tt8046498,movie,N/A,N/A,N/A,N/A,True,7.9/10,N/A,N/A


In [10]:
result[result['Rating'] != result['imdbRating']][['Rating', 'imdbRating']]

# drop Type as it is always Movie, Response (true), Website as it's N/A, Ratings as it's separated into columns
result.drop(['imdbRating', 'imdbID', 'Type', 'Response', 'Website', 'Ratings'], axis=1, inplace=True)

result

,Rank,Id,Image,Titlemeter count,Titlemeter,Rating,Rating based on,Title,Year,Rated,...,Awards,Poster,Metascore,imdbVotes,DVD,BoxOffice,Production,Internet Movie Database,Metacritic,Rotten Tomatoes
0,1,tt11161474,https://m.media-amazon.com/images/M/MV5BMGE3Mz...,4,up,7.1,"7.1 based on 15,722 user ratings",Pieces of a Woman,2020,R,...,2 wins & 28 nominations.,https://m.media-amazon.com/images/M/MV5BMGE3Mz...,66,"13,523",N/A,N/A,N/A,N/A,66/100,N/A
1,2,tt2948372,https://m.media-amazon.com/images/M/MV5BZGE1MD...,1,down,8.1,"8.1 based on 139,955 user ratings",Soul,2020,PG,...,5 wins & 18 nominations.,https://m.media-amazon.com/images/M/MV5BZGE1MD...,86,662,N/A,N/A,N/A,N/A,86/100,N/A
2,3,tt7126948,https://m.media-amazon.com/images/M/MV5BNWY2NW...,1,down,5.4,"5.4 based on 137,696 user ratings",Wonder Woman 1984,2020,PG-13,...,5 nominations.,https://m.media-amazon.com/images/M/MV5BNWY2NW...,60,"129,163",N/A,"$32,600,000","DC Comics, Warner Bros., DC Entertainment, Atl...",N/A,60/100,N/A
3,4,tt6723592,https://m.media-amazon.com/images/M/MV5BYzg0NG...,1,down,7.5,"7.5 based on 271,311 user ratings",Tenet,2020,PG-13,...,12 wins & 35 nominations.,https://m.media-amazon.com/images/M/MV5BYzg0NG...,69,"262,974",N/A,"$57,929,000","Syncopy, Warner Bros. Pictures",N/A,N/A,N/A
4,5,tt9130508,https://m.media-amazon.com/images/M/MV5BYTY3Nz...,116,up,N/A,N/A,Cherry,2021,R,...,N/A,https://m.media-amazon.com/images/M/MV5BOWE1Nj...,N/A,N/A,N/A,N/A,N/A,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,tt0109830,https://m.media-amazon.com/images/M/MV5BNWIwOD...,4,down,8.8,"8.8 based on 1,804,075 user ratings",Forrest Gump,1994,PG-13,...,Won 6 Oscars. Another 44 wins & 75 nominations.,https://m.media-amazon.com/images/M/MV5BNWIwOD...,82,"1,801,266",N/A,"$330,455,270",Paramount Pictures,N/A,82/100,N/A
96,97,tt4263482,https://m.media-amazon.com/images/M/MV5BMTUyNz...,51,up,6.9,"6.9 based on 200,714 user ratings",The Witch,2015,R,...,43 wins & 68 nominations.,https://m.media-amazon.com/images/M/MV5BMTUyNz...,83,"201,120",N/A,"$25,138,705",N/A,N/A,83/100,N/A
97,98,tt4154756,https://m.media-amazon.com/images/M/MV5BMjMxNj...,35,down,8.4,"8.4 based on 830,402 user ratings",Avengers: Infinity War,2018,PG-13,...,Nominated for 1 Oscar. Another 46 wins & 75 no...,https://m.media-amazon.com/images/M/MV5BMjMxNj...,68,"828,791",N/A,"$678,815,482",Marvel Studios,N/A,68/100,N/A
98,99,tt8046498,https://m.media-amazon.com/images/M/MV5BZmIzOW...,845,up,5.9,5.9 based on 669 user ratings,Don't Tell a Soul,2020,R,...,N/A,https://m.media-amazon.com/images/M/MV5BZmIzOW...,N/A,55,N/A,N/A,N/A,7.9/10,N/A,N/A


In [11]:
result.columns, len(result.columns)

(Index(['Rank', 'Id', 'Image', 'Titlemeter count', 'Titlemeter', 'Rating',
        'Rating based on', 'Title', 'Year', 'Rated', 'Released', 'Runtime',
        'Genre', 'Director', 'Writer', 'Actors', 'Plot', 'Language', 'Country',
        'Awards', 'Poster', 'Metascore', 'imdbVotes', 'DVD', 'BoxOffice',
        'Production', 'Internet Movie Database', 'Metacritic',
        'Rotten Tomatoes'],
       dtype='object'),
 29)

In [14]:
filename = 'joined_data.csv'
files_present = glob.glob(filename)

# if no matching files, write to csv, if there are matching files, print statement
if not files_present:
    result.to_csv(filename)
    print("Sucessfully written into joined_data.csv")
else:
    print('WARNING: This file already exists!')

Sucessfully written into joined_data.csv
